In [56]:
import pysftp
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import time
from time import sleep
import os
import shutil 
import sys
from tqdm import tqdm
now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H%M%S"))

In [62]:
#Creating Employee Master Backup
print("Creating Employee Master Backup...") 
# for i in tqdm(range(10)):
#     sleep(0.1)
src = Path("../Resources/Source_data/master.csv")
dst = Path("../Resources/Source_data/Archive/MASTER_BACKUP.csv")
shutil.copy(src, dst)
print("Employee Master Backup Complete")

 10%|█         | 1/10 [00:00<00:01,  8.41it/s]

Creating Employee Master Backup...


100%|██████████| 10/10 [00:01<00:00,  8.81it/s]

Employee Master Backup Complete


In [ ]:
#Downloading Updates
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None
# sftp = pysftp.Connection('sftp.talentreef.com', username=hidden, password=hidden, cnopts=cnopts)
# sftp.get('/outbound/pde/TCF_18703/TCF_Terminations/TCF_Terminations20210802-1300-23.csv')
# print("Process Complete")

#Updating Employee Master



In [11]:
#Loading Employee Data
employee_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), header = 0, usecols = ["EMP_NO","PAYCODE","PAYRATE"])
hour_data = pd.read_csv(Path("../Resources/Source_data/hours.csv")) 
employee_ach_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), header = 0, usecols = ["EMP_NO","FIRST_NAME","LAST_NAME","BANK_RTN","BANK_ACC"], dtype = {"BANK_RTN" : str, "BANK_ACC" : str})
employee_data_updates = pd.read_csv(Path("../Resources/Source_data/updates.csv"), header = 0, usecols = ["EMP_NO","PAYCODE","PAYRATE"])   

In [12]:
#Calculating Hourly Wages
hourly_emp_data = employee_data.loc[employee_data["PAYCODE"] == "Hourly"]
hourly_grid = pd.merge(hourly_emp_data, hour_data, how = "inner", on = "EMP_NO")
hourly_grid["WAGES"] = np.where(hourly_grid["HOURS"] > 40, 
    ((hourly_grid["HOURS"]-40)*1.5*hourly_grid["PAYRATE"])+ (hourly_grid["PAYRATE"]*40), 
    (hourly_grid["HOURS"] * hourly_grid["PAYRATE"]))
hourly_grid.drop(["PAYCODE","PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,WAGES
0,101001,847.5
1,101003,1400.0
2,101005,1800.0
3,101006,1062.5
4,101007,825.0
5,101009,600.0
6,101010,680.0
7,101011,1400.0


In [13]:
#Calculating Salary Wages
salary_grid = employee_data.loc[employee_data["PAYCODE"] == "Salary"]
salary_grid["WAGES"] = salary_grid["PAYRATE"] / 52
salary_grid.drop(["PAYCODE","PAYRATE"], axis=1).round(2)

<ipython-input-13-93df724939da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_grid["WAGES"] = salary_grid["PAYRATE"] / 52


,EMP_NO,WAGES
0,101000,2307.69
2,101002,1442.31
4,101004,1442.31
8,101008,1826.92


In [14]:
#Combining Salary and Hourly Wages
all_wages = hourly_grid.append(salary_grid, ignore_index=True, verify_integrity= True)
all_wages.drop(["PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,PAYCODE,WAGES
0,101001,Hourly,847.50
1,101003,Hourly,1400.00
2,101005,Hourly,1800.00
3,101006,Hourly,1062.50
4,101007,Hourly,825.00
5,101009,Hourly,600.00
6,101010,Hourly,680.00
7,101011,Hourly,1400.00
8,101000,Salary,2307.69
9,101002,Salary,1442.31


In [15]:
#Creating ACH File
employee_ach_data["NAME"] =  employee_ach_data["FIRST_NAME"].str.upper() + [" "] + employee_ach_data["LAST_NAME"].str.upper()
ach_data = pd.merge(all_wages, employee_ach_data, how="inner", on = "EMP_NO")
ach_data.drop(["FIRST_NAME","LAST_NAME","PAYCODE", "PAYRATE", "HOURS"], axis=1).round(2)

,EMP_NO,WAGES,BANK_RTN,BANK_ACC,NAME
0,101001,847.50,124085244,1321541313,JAMEL BARDNEY
1,101003,1400.00,091000019,21546931354,IVONNE PERALTA VELAZQUEZ
2,101005,1800.00,073905912,2136564123,THOMAS HENDRICKS
3,101006,1062.50,124085244,16848643139846,MARCELL MITCHELL
4,101007,825.00,041215663,164864318771,VALARIE LOPEZ
5,101009,600.00,124085244,569746414,DEWAYNE CHESTER
6,101010,680.00,091408734,16874136,DIAMOND ZAMORA
7,101011,1400.00,256074974,1235483141,MALIK CHARLESTON
8,101000,2307.69,256074974,1235483141,ERIC LANE
9,101002,1442.31,124085244,1215445,CHUDIER CHUOL
